## Firehose Dataset Parser

Official website of Firehose http://firebrowse.org/
Official website of Broad Institute from where we can download all the datasets http://gdac.broadinstitute.org/#

This tool automate the downloading of datasets from the broad institute official site. We have to choose the cancer tyoe from the available cancer types and it will download the datasets of corresponding cancer type. 

In [172]:
# Import all the necessary libraries
import ipywidgets as widgets
from IPython.display import display
import requests
import os

In [173]:
# All available cancer types in Broad Institutes official site
cancers = [('Adrenocortical carcinoma', 'ACC'),
('Bladder Urothelial Carcinoma', 'BLCA'),
('Breast invasive carcinoma', 'BRCA'),
('Cervical squamous cell carcinoma and endocervical adenocarcinoma', 'CESC'),
('Cholangiocarcinoma', 'CHOL'),
('Colon adenocarcinoma', 'COAD'),
('Colorectal adenocarcinoma', 'COADREAD'),
('Lymphoid Neoplasm Diffuse Large B-cell Lymphoma', 'DLBC'),
('Esophageal carcinoma', 'ESCA'),
('FFPE Pilot Phase II', 'FPPP'),
('Glioblastoma multiforme', 'GBM'),
('Glioma', 'GBMLGG'),
('Head and Neck squamous cell carcinoma', 'HNSC'),
('Kidney Chromophobe', 'KICH'),
('Pan-kidney cohort (KICH+KIRC+KIRP)', 'KIPAN'),
('Kidney renal clear cell carcinoma', 'KIRC'),
('Kidney renal papillary cell carcinoma', 'KIRP'),
('Acute Myeloid Leukemia', 'LAML'),
('Brain Lower Grade Glioma', 'LGG'),
('Liver hepatocellular carcinoma', 'LIHC'),
('Lung adenocarcinoma', 'LUAD'),
('Lung squamous cell carcinoma', 'LUSC'),
('Mesothelioma', 'MESO'),
('Ovarian serous cystadenocarcinoma', 'OV'),
('Pancreatic adenocarcinoma', 'PAAD'),
('Pheochromocytoma and Paraganglioma', 'PCPG'),
('Prostate adenocarcinoma', 'PRAD'),
('Rectum adenocarcinoma', 'READ'),
('Sarcoma', 'SARC'),
('Skin Cutaneous Melanoma', 'SKCM'),
('Stomach adenocarcinoma', 'STAD'),
('Stomach and Esophageal carcinoma', 'STES'),
('Testicular Germ Cell Tumors', 'TGCT'),
('Thyroid carcinoma', 'THCA'),
('Thymoma', 'THYM'),
('Uterine Corpus Endometrial Carcinoma', 'UCEC'),
('Uterine Carcinosarcoma', 'UCS'),
('Uveal Melanoma', 'UVM')]

In [174]:
# Function for safe directory creation
def create_dir(path):
    try:
        os.mkdir(path)
    except Exception as ex:
        print('Directory already exist!')

In [175]:
# Prepare directories here
# Create /data directory if not exist. All datasets will be placed in this directory
create_dir('data')

Directory already exist!


### download_data(url) function downloads file using requests library of python
All data files are downloaded in the /data directory

In [176]:
# Helper function to download file and save into /data directory
def download_data(url):
    try:
        sp_url = url.split('/') # Url Array
        
        # Create directory for particular cancer
        cancer_path = 'data/' + sp_url[-3]
        print(cancer_path)
        create_dir(cancer_path)
        
        # Request instance for downloading file
        req = requests.get(url, allow_redirects=True)
        
        # Format file name
        file_name = sp_url[-3] + '__' + sp_url[-2] + '__' + sp_url[-1] 
    
        # Setup file path
        data_path = cancer_path + '/' + file_name
        print(data_path)
        open(data_path, 'wb').write(req.content)
        
    except Exception as ex:
        print("File not found!")

### There was a common pattern in download url
Using those common pattern of the download url, we can construct a download url for different datasets

In [177]:
def download_copy_number(selected_cancer):
    dataset_url = "http://gdac.broadinstitute.org/runs/analyses__2016_01_28/data/" + selected_cancer + "-TP/20160128/gdac.broadinstitute.org_" + selected_cancer + "-TP.CopyNumber_Gistic2.Level_4.2016012800.0.0.tar.gz"

    del_url = "http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/" + selected_cancer + "-TP/CopyNumber_Gistic2/del_genes.conf_99.txt"
    amp_url = "http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/" + selected_cancer + "-TP/CopyNumber_Gistic2/amp_genes.conf_99.txt"
    arm_url = "http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/" + selected_cancer + "-TP/CopyNumber_Gistic2/broad_significance_results.txt"
    broad_arm_url = "http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/" + selected_cancer + "-TP/CopyNumber_Gistic2/broad_values_by_arm.txt"

    
    print(dataset_url)
    print(del_url)
    print(amp_url)
    print(arm_url)
    print(broad_arm_url)

    download_data(del_url)
    download_data(amp_url)
    download_data(arm_url)
    download_data(broad_arm_url)
    download_data(dataset_url)
    

### Mutation Analysis datasets also has similar type of symmetricity

In [178]:
def download_mutsig(selected_cancer):
    dataset_url = "http://gdac.broadinstitute.org/runs/analyses__2016_01_28/data/" + selected_cancer + "-TP/20160128/gdac.broadinstitute.org_" + selected_cancer + "-TP.MutSigNozzleReport2CV.Level_4.2016012800.0.0.tar.gz"
    mut_url = "http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/" + selected_cancer + "-TP/MutSigNozzleReport2CV/sig_genes.txt"
    
    print(dataset_url)
    print(mut_url)

    download_data(mut_url)
    download_data(dataset_url)
    

### on_change(change) function is a event trigger function
This function triggers when the user change the type of cancer in dropdown menu. As soon as user change the cancer type from the dropdown this function downloads datasets.

In [179]:
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_cancer = change['new']
        print("changed to %s" % selected_cancer)
        
        download_copy_number(selected_cancer)
        download_mutsig(selected_cancer) 

Now let's create a dropdown menu for all the types of cancers available in broad firehose website

In [180]:
w = widgets.Dropdown(
    options = cancers,
    value='GBM',
    description='Cancers DropDown List',
)

w.observe(on_change)

display(w)

Dropdown(description='Cancers DropDown List', index=10, options=(('Adrenocortical carcinoma', 'ACC'), ('Bladde…

changed to BRCA
http://gdac.broadinstitute.org/runs/analyses__2016_01_28/data/BRCA-TP/20160128/gdac.broadinstitute.org_BRCA-TP.CopyNumber_Gistic2.Level_4.2016012800.0.0.tar.gz
http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/BRCA-TP/CopyNumber_Gistic2/del_genes.conf_99.txt
http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/BRCA-TP/CopyNumber_Gistic2/amp_genes.conf_99.txt
http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/BRCA-TP/CopyNumber_Gistic2/broad_significance_results.txt
http://gdac.broadinstitute.org/runs/analyses__2016_01_28/reports/cancer/BRCA-TP/CopyNumber_Gistic2/broad_values_by_arm.txt
data/BRCA-TP
data/BRCA-TP/BRCA-TP__CopyNumber_Gistic2__del_genes.conf_99.txt
data/BRCA-TP
Directory already exist!
data/BRCA-TP/BRCA-TP__CopyNumber_Gistic2__amp_genes.conf_99.txt
data/BRCA-TP
Directory already exist!
data/BRCA-TP/BRCA-TP__CopyNumber_Gistic2__broad_significance_results.txt
data/BRCA-TP
Directory already exist!
d

## Using panda, and numpy

In [171]:
import pandas as pd
import numpy as np

In [161]:
df = pd.read_csv('data/BRCA-TP/BRCA-TP__MutSigNozzleReport2CV__sig_genes.txt', delimiter="\t")
# df = pd.read_fwf('cancers.txt')
df.head()


,rank,gene,longname,codelen,nnei,nncd,nsil,nmis,nstp,nspl,nind,nnon,npat,nsite,pCV,pCL,pFN,p,q
0,1,PIK3CA,"phosphoinositide-3-kinase, catalytic, alpha po...",3287,13,0,6,338,0,6,7,351,316,53,1.000000e-16,0.000010,0.00001,1.000000e-16,4.567000e-13
1,2,TP53,tumor protein p53,1890,0,0,4,173,45,23,59,300,297,161,1.000000e-16,0.000010,0.00001,1.000000e-16,4.567000e-13
2,3,GATA3,GATA binding protein 3,1351,20,0,2,9,2,1,89,101,97,56,1.557174e-15,0.000010,0.36125,1.000000e-16,4.567000e-13
3,4,MAP3K1,mitogen-activated protein kinase kinase kinase 1,4615,7,0,1,16,14,4,64,98,71,88,1.000000e-16,0.000020,0.92878,1.000000e-16,4.567000e-13
4,5,CDH1,"cadherin 1, type 1, E-cadherin (epithelial)",2709,6,0,1,16,29,12,52,109,107,90,1.233779e-15,0.000533,0.67910,2.220446e-16,8.112622e-13
